# Notebook to create netcdf file with integrated parameters (old approach)

## Importing

In [5]:
import xarray as xr
import numpy as np
import pandas as pd

import os

from tqdm import tqdm


## Datasets Preparation

In [6]:
def datasets_preparation():

    ds_name = ('/results2/SalishSea/nowcast-green.202111/' + i + '/SalishSea_1d_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]])+str(i[0:2]) + '_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]]) + str(i[0:2]) + '_grid_T.nc')
    ds_bio_name = ('/results2/SalishSea/nowcast-green.202111/' + i + '/SalishSea_1d_'  + '20' + str(i[5:7]) + str(dict_month[i[2:5]])+str(i[0:2]) + '_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]]) + str(i[0:2]) + '_biol_T.nc')

    ds = xr.open_dataset (ds_name)
    ds_bio = xr.open_dataset (ds_bio_name)

    temp_i1 = ((ds.votemper.where(mask==1)[0,0:15] * ds.e3t.where(mask==1)
        [0,0:15]).sum('deptht', skipna = True, min_count = 15) / mesh.gdepw_0[0,15])
    temp_i2 = ((ds.votemper.where(mask==1)[0,15:27] * ds.e3t.where(mask==1)
        [0,15:27]).sum('deptht', skipna = True, min_count = 12) / (mesh.gdepw_0
        [0,27] - mesh.gdepw_0[0,14]))

    saline_i1 = ((ds.vosaline.where(mask==1)[0,0:15] * ds.e3t.where(mask==1)
        [0,0:15]).sum('deptht', skipna = True, min_count = 15) / mesh.gdepw_0[0,15])
    saline_i2 = ((ds.vosaline.where(mask==1)[0,15:27] * ds.e3t.where(mask==1)
        [0,15:27]).sum('deptht', skipna = True, min_count = 12) / (mesh.gdepw_0
        [0,27] - mesh.gdepw_0[0,14]))

    sil_i = ((ds_bio.silicon.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    nitr_i = ((ds_bio.nitrate.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    ammo_i = ((ds_bio.ammonium.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    diat_i = ((ds_bio.diatoms.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    flag_i = ((ds_bio.flagellates.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    micro_i = ((ds_bio.microzooplankton.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    meso_i = ((ds_bio.mesozooplankton.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    return (temp_i1, temp_i2, saline_i1, saline_i2, sil_i, nitr_i, ammo_i, diat_i, flag_i, micro_i, meso_i)


## Datasets Preparation 2

In [7]:
def datasets_preparation2(description,name,units):

    dataset = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'],
        attrs=dict(description=description,
        long_name =name,
        units=units))
        
    return (dataset)

## File Creation

In [8]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/analysis-ilias/notebooks/integrated_original.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})


## Data Collection

In [9]:
dict_month = {'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'}

path = os.listdir('/results2/SalishSea/nowcast-green.202111/')

folders = [x for x in path if ((x[2:5]=='mar' or x[2:5]=='apr' or (x[2:5]=='feb' and x[0:2] > '14')) and (x[5:7]<'24'))]
indx_dates=(np.argsort(pd.to_datetime(folders, format="%d%b%y")))
folders = [folders[i] for i in indx_dates]

# Open the mesh mask
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
mask = mesh.tmask.to_numpy()

coords = dict(time_counter=ds.time_counter, y=ds.y, x=ds.x) 

temp_i1_all = datasets_preparation2('Conservative Temperature (0m - 15m)', 
    'Conservative Temperature', '[degree_C m]')
temp_i2_all = datasets_preparation2('Conservative Temperature (15m - 100m)', 
    'Conservative Temperature', '[degree_C m]')
saline_i1_all = datasets_preparation2('Reference Salinity (0m - 15m)',
    'Reference Salinity', '[g kg-1 m]')
saline_i2_all = datasets_preparation2('Reference Salinity (15m - 100m)',
    'Reference Salinity', '[g kg-1 m]')

sil_i_all = datasets_preparation2('Silicon Concentration', 'Silicon', '[mmol m-2]')
nitr_i_all = datasets_preparation2('Nitrate Concentration', 'Nitrate', '[mmol m-2]')
ammo_i_all = datasets_preparation2('Ammonium Concentration', 'Ammonium', '[mmol m-2]')

diat_i_all = datasets_preparation2('Diatom Concentration', 'Diatom', '[mmol m-2]')
flag_i_all = datasets_preparation2('Flagellate Concentration', 'Flagellate', '[mmol m-2]')

micro_i_all = datasets_preparation2('Microzooplankton Concentration', 
    'Microzooplankton', '[mmol m-2]')
meso_i_all = datasets_preparation2('Mesozooplankton Concentration', 
    'Mesozooplankton', '[mmol m-2]')

for i in tqdm(enumerate(folders)):

    temp_i1, temp_i2, saline_i1, saline_i2, sil_i, nitr_i, ammo_i, diat_i, flag_i, micro_i, meso_i = datasets_preparation()

    temp_i1_all[i[0]] = temp_i1
    temp_i2_all[i[0]] = temp_i2
    saline_i1_all[i[0]] = saline_i1
    saline_i2_all[i[0]] = saline_i2

    sil_i_all[i[0]] = sil_i
    nitr_i_all[i[0]] = nitr_i
    ammo_i_all[i[0]] = ammo_i

    diat_i_all[i[0]] = diat_i
    flag_i_all[i[0]] = flag_i

    micro_i_all[i[0]] = micro_i
    meso_i_all[i[0]] = meso_i


# Calling file creation

file_creation(temp_i1_all, 'Temperature_(0m-15m)')
file_creation(temp_i2_all, 'Temperature_(15m-100m)')
file_creation(saline_i1_all, 'Salinity_(0m-15m)')
file_creation(saline_i2_all, 'Salinity_(15m-100m)')

file_creation(sil_i_all, 'Silicon')
file_creation(nitr_i_all, 'Nitrate')
file_creation(ammo_i_all, 'Ammonium')

file_creation(diat_i_all, 'Diatom')
file_creation(flag_i_all, 'Flagellate')

file_creation(micro_i_all, 'Microzooplankton')
file_creation(meso_i_all, 'Mesozooplankton')


NameError: name 'ds' is not defined